In [162]:
pip install pandas sqlalchemy

In [163]:
import pandas as pd
import json
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from datetime import datetime
import sqlite3

In [164]:
# Function to load JSON data line by line
def load_json_line_by_line(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():  # Ignore empty lines
                data.append(json.loads(line))
    return data

# Load data
receipts_data = load_json_line_by_line('receipts.json')
brands_data = load_json_line_by_line('brands.json')
users_data = load_json_line_by_line('users.json')

# Convert to DataFrame
df_receipts = pd.json_normalize(receipts_data)
df_brands = pd.json_normalize(brands_data)
df_users = pd.json_normalize(users_data)

In [165]:
print(df_receipts.head())
print(df_brands.head())
print(df_users.head())

   bonusPointsEarned                            bonusPointsEarnedReason  \
0              500.0  Receipt number 2 completed, bonus point schedu...   
1              150.0  Receipt number 5 completed, bonus point schedu...   
2                5.0                         All-receipts receipt bonus   
3                5.0                         All-receipts receipt bonus   
4                5.0                         All-receipts receipt bonus   

  pointsEarned  purchasedItemCount  \
0        500.0                 5.0   
1        150.0                 2.0   
2            5                 1.0   
3          5.0                 4.0   
4          5.0                 2.0   

                              rewardsReceiptItemList rewardsReceiptStatus  \
0  [{'barcode': '4011', 'description': 'ITEM NOT ...             FINISHED   
1  [{'barcode': '4011', 'description': 'ITEM NOT ...             FINISHED   
2  [{'needsFetchReview': False, 'partnerItemId': ...             REJECTED   
3  [{'barcod

In [166]:
df_1=pd.DataFrame(df_receipts['rewardsReceiptItemList'])
df_1

,rewardsReceiptItemList
0,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
1,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
2,"[{'needsFetchReview': False, 'partnerItemId': ..."
3,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
4,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
...,...
1114,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1115,NaN
1116,NaN
1117,"[{'barcode': 'B076FJ92M4', 'description': 'mue..."


In [167]:
# Count NaN values in each column
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

print(df_receipts['bonusPointsEarned'].mean())

Count of NaN values in each column:
bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
_id.$oid                     0
createDate.$date             0
dateScanned.$date            0
finishedDate.$date         551
modifyDate.$date             0
pointsAwardedDate.$date    582
purchaseDate.$date         448
dtype: int64
(1119, 15)
238.8933823529412


In [168]:
rejected_receipts = df_receipts[df_receipts['rewardsReceiptStatus'] == 'REJECTED']
rejected_receipts

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
13,750.0,"Receipt number 1 completed, bonus point schedu...",750.0,11.0,"[{'barcode': '075925306254', 'competitiveProdu...",REJECTED,1.00,5f9c74f7c88c1415cbddb839,5f9c74f70a7214ad07000037,1604089079000,1604089079000,NaN,1609687494000,1.604089e+12,1.604003e+12
28,NaN,NaN,NaN,3.0,"[{'deleted': True, 'description': 'DELETED ITE...",REJECTED,3.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d40a7214ada1000562,1609687508000,1609687508000,NaN,1609687508000,NaN,1.509322e+12
62,750.0,"Receipt number 1 completed, bonus point schedu...",750.0,2.0,[{'description': 'mueller austria hypergrind p...,REJECTED,34.96,5fbc35711d967d1222cbfefc,5ff4a4ca0a7214ada10005d0,1609868490000,1609868490000,NaN,1609868492000,NaN,1.599955e+12
175,NaN,NaN,NaN,0.0,NaN,REJECTED,0.00,5ff8da28b3348b11c9337ac6,5ff8da570a720f05c5000015,1610144343000,1610144343000,NaN,1610144344000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603c7c6c0a7217c72c0003b3,1614576748000,1614576748000,NaN,1614576749000,NaN,1.597622e+12
1112,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603c3d240a720fde10000373,1614560548000,1614560548000,NaN,1614560549000,NaN,1.597622e+12
1113,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc2bc0a720fde100003e9,1614594748000,1614594748000,NaN,1614594749000,NaN,1.597622e+12
1114,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc0630a720fde100003e6,1614594147000,1614594147000,NaN,1614594148000,NaN,1.597622e+12


In [169]:
df_receipts[['dateScanned.$date','purchaseDate.$date']]

,dateScanned.$date,purchaseDate.$date
0,1609687531000,1.609632e+12
1,1609687483000,1.609601e+12
2,1609687537000,1.609632e+12
3,1609687534000,1.609632e+12
4,1609687506000,1.609601e+12
...,...,...
1114,1614594147000,1.597622e+12
1115,1614613361873,NaN
1116,1614607657664,NaN
1117,1614604048000,1.597622e+12


In [170]:
df_receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [171]:
nan_bonus_points = df_receipts[df_receipts['bonusPointsEarned'].isna()]

# Display the rows
print(nan_bonus_points)

      bonusPointsEarned bonusPointsEarnedReason pointsEarned  \
15                  NaN                     NaN          NaN   
28                  NaN                     NaN          NaN   
32                  NaN                     NaN        500.0   
35                  NaN                     NaN        250.0   
52                  NaN                     NaN        350.0   
...                 ...                     ...          ...   
1110                NaN                     NaN          NaN   
1111                NaN                     NaN          NaN   
1115                NaN                     NaN          NaN   
1116                NaN                     NaN          NaN   
1118                NaN                     NaN          NaN   

      purchasedItemCount                             rewardsReceiptItemList  \
15                   0.0  [{'needsFetchReview': True, 'needsFetchReviewR...   
28                   3.0  [{'deleted': True, 'description': 'DELETED ITE.

In [172]:
df_receipts.columns

Index(['bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', '_id.$oid', 'createDate.$date',
       'dateScanned.$date', 'finishedDate.$date', 'modifyDate.$date',
       'pointsAwardedDate.$date', 'purchaseDate.$date'],
      dtype='object')

In [173]:
print(df_receipts['rewardsReceiptItemList'])
print(df_receipts['purchasedItemCount'])
print(df_receipts['totalSpent'])
print(df_receipts['dateScanned.$date'])

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
2       [{'needsFetchReview': False, 'partnerItemId': ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
1114    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1115                                                  NaN
1116                                                  NaN
1117    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 1119, dtype: object
0       5.0
1       2.0
2       1.0
3       4.0
4       2.0
       ... 
1114    2.0
1115    NaN
1116    NaN
1117    2.0
1118    NaN
Name: purchasedItemCount, Length: 1119, dtype: float64
0       26.00
1       11.00
2       10.00
3       28.00
4        1.00
        ...  
1114    34.96
1115      NaN
1116 

In [174]:
specific_oids = ['6009eb000a7214ada2000003', '600aff160a720f053500000c', '601b32160a7214ad28000221']

# Filter rows with specific OIDs and NaN in rewardsReceiptItemList
filtered_rows = df_receipts[df_receipts['_id.$oid'].isin(specific_oids) & df_receipts['rewardsReceiptItemList'].isna()]
filtered_rows

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
396,250.0,"Receipt number 3 completed, bonus point schedu...",250.0,0.0,NaN,FINISHED,0.00,6009e60450b3311194385009,6009eb000a7214ada2000003,1611262720000,1611262720000,1.611263e+12,1611262755000,1.611263e+12,1.611187e+12
424,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,0.0,NaN,FINISHED,0.00,600afb2a7d983a124e9aded0,600aff160a720f053500000c,1611333398000,1611333398000,1.611333e+12,1611333433000,1.611333e+12,1.611274e+12
727,NaN,NaN,NaN,0.0,NaN,REJECTED,0.00,5a43c08fe4b014fd6b6a0612,601b32160a7214ad28000221,1612395030000,1612395030000,NaN,1612395031000,NaN,1.546474e+12


In [175]:
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

Count of NaN values in each column:
bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
_id.$oid                     0
createDate.$date             0
dateScanned.$date            0
finishedDate.$date         551
modifyDate.$date             0
pointsAwardedDate.$date    582
purchaseDate.$date         448
dtype: int64
(1119, 15)


In [176]:
df_receipts = df_receipts.drop(columns=['bonusPointsEarned', 'bonusPointsEarnedReason','pointsEarned','pointsAwardedDate.$date','purchaseDate.$date','finishedDate.$date',])

In [177]:
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

Count of NaN values in each column:
purchasedItemCount        484
rewardsReceiptItemList    440
rewardsReceiptStatus        0
totalSpent                435
userId                      0
_id.$oid                    0
createDate.$date            0
dateScanned.$date           0
modifyDate.$date            0
dtype: int64
(1119, 9)


In [178]:
nan_purchased_item_count_rows = df_receipts[df_receipts['purchasedItemCount'].isna()]

print("Rows where purchasedItemCount is NaN:")
print(nan_purchased_item_count_rows)

Rows where purchasedItemCount is NaN:
      purchasedItemCount                             rewardsReceiptItemList  \
71                   NaN                                                NaN   
93                   NaN                                                NaN   
149                  NaN                                                NaN   
210                  NaN  [{'description': 'flipbelt level terrain waist...   
211                  NaN  [{'description': 'flipbelt level terrain waist...   
...                  ...                                                ...   
1110                 NaN                                                NaN   
1111                 NaN                                                NaN   
1115                 NaN                                                NaN   
1116                 NaN                                                NaN   
1118                 NaN                                                NaN   

     rewardsR

In [179]:
for index, row in nan_purchased_item_count_rows.iterrows():
    print(f"Receipt ID: {row['_id.$oid']}")
    rewards_receipt_items = row['rewardsReceiptItemList']
    if isinstance(rewards_receipt_items, list):
        for item in rewards_receipt_items:
            print(f"  Description: {item.get('description', 'N/A')}")
            print(f"  Final Price: {item.get('finalPrice', 'N/A')}")
            print(f"  Quantity Purchased: {item.get('quantityPurchased', 'N/A')}")
    else:
        print("  No items in rewardsReceiptItemList")
    print()

Receipt ID: 5ff475820a7214ada10005cf
  No items in rewardsReceiptItemList

Receipt ID: 5ff5ecb90a7214ada10005f9
  No items in rewardsReceiptItemList

Receipt ID: 5ff726860a720f05230005ec
  No items in rewardsReceiptItemList

Receipt ID: 5ffe1cbe0a7214ad28002843
  Description: flipbelt level terrain waist pouch, neon yellow, large/32-35
  Final Price: 28.57
  Quantity Purchased: 1

Receipt ID: 5ffce7db0a720f051500236e
  Description: flipbelt level terrain waist pouch, neon yellow, large/32-35
  Final Price: 28.57
  Quantity Purchased: 1

Receipt ID: 5ffce8570a7214ad4e003e6f
  No items in rewardsReceiptItemList

Receipt ID: 5ffce8310a7214ad4e003797
  Description: flipbelt level terrain waist pouch, neon yellow, large/32-35
  Final Price: 28.57
  Quantity Purchased: 1

Receipt ID: 5ffe19d90a7214ad28000e62
  Description: flipbelt level terrain waist pouch, neon yellow, large/32-35
  Final Price: 28.57
  Quantity Purchased: 1

Receipt ID: 5ffe1d030a720f05ac002c9e
  Description: flipbelt lev

In [180]:
#nan_rows = df_receipts[df_receipts['purchasedItemCount'].isna() & df_receipts['rewardsReceiptItemList'].isna()]

# Print the resulting rows
#print(nan_rows)
df_receipts = df_receipts.dropna(subset=['purchasedItemCount', 'rewardsReceiptItemList'], how='all')


In [181]:
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

Count of NaN values in each column:
purchasedItemCount        49
rewardsReceiptItemList     5
rewardsReceiptStatus       0
totalSpent                 0
userId                     0
_id.$oid                   0
createDate.$date           0
dateScanned.$date          0
modifyDate.$date           0
dtype: int64
(684, 9)


In [182]:
ids_to_update = ['5ffe1cbe0a7214ad28002843', '601431e40a7214ad50000041']

# Update the purchasedItemCount for the specified IDs
df_receipts.loc[df_receipts['_id.$oid'].isin(ids_to_update), 'purchasedItemCount'] = 1


In [183]:
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

Count of NaN values in each column:
purchasedItemCount        47
rewardsReceiptItemList     5
rewardsReceiptStatus       0
totalSpent                 0
userId                     0
_id.$oid                   0
createDate.$date           0
dateScanned.$date          0
modifyDate.$date           0
dtype: int64
(684, 9)


In [184]:
df_receipts = df_receipts.dropna(subset=['purchasedItemCount'])

In [185]:
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

Count of NaN values in each column:
purchasedItemCount        0
rewardsReceiptItemList    5
rewardsReceiptStatus      0
totalSpent                0
userId                    0
_id.$oid                  0
createDate.$date          0
dateScanned.$date         0
modifyDate.$date          0
dtype: int64
(637, 9)


In [186]:
nan_rewards_receipt_item_list_rows = df_receipts[df_receipts['rewardsReceiptItemList'].isna()]
print("Rows where rewardsReceiptItemList is NaN:")
nan_rewards_receipt_item_list_rows

Rows where rewardsReceiptItemList is NaN:


,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,modifyDate.$date
175,0.0,NaN,REJECTED,0.00,5ff8da28b3348b11c9337ac6,5ff8da570a720f05c5000015,1610144343000,1610144343000,1610144344000
396,0.0,NaN,FINISHED,0.00,6009e60450b3311194385009,6009eb000a7214ada2000003,1611262720000,1611262720000,1611262755000
424,0.0,NaN,FINISHED,0.00,600afb2a7d983a124e9aded0,600aff160a720f053500000c,1611333398000,1611333398000,1611333433000
656,0.0,NaN,REJECTED,0.00,60182f1dc8b50e11d84548c4,60182f390a7214ad500002b8,1612197689000,1612197689000,1612197690000
727,0.0,NaN,REJECTED,0.00,5a43c08fe4b014fd6b6a0612,601b32160a7214ad28000221,1612395030000,1612395030000,1612395031000


In [187]:
df_receipts = df_receipts.dropna(subset=['rewardsReceiptItemList'])

In [188]:
nan_counts = df_receipts.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_receipts.shape)

Count of NaN values in each column:
purchasedItemCount        0
rewardsReceiptItemList    0
rewardsReceiptStatus      0
totalSpent                0
userId                    0
_id.$oid                  0
createDate.$date          0
dateScanned.$date         0
modifyDate.$date          0
dtype: int64
(632, 9)


In [189]:
df_brands

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs,NaN
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs,LIPTON TEA Leaves


In [190]:
filtered_brands = df_brands[df_brands['name'].str.startswith('test')]
filtered_brands

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
5,511111719885,Baking,BAKING,test brand @1612366146091,False,601ac142be37ce2ead43755b,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146091
...,...,...,...,...,...,...,...,...,...
1152,511111715559,Baking,BAKING,test brand @1597935986434,NaN,5f3e9172be37ce5a0e01b955,5f3e9172be37ce5a0e01b952,Cogs,TEST BRANDCODE @1597935986434
1158,511111716648,Baking,BAKING,test brand @1600291349042,NaN,5f628215be37ce78e6e2efab,5f628214be37ce78e6e2efaa,Cogs,TEST BRANDCODE @1600291349043
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs,NaN
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217


In [191]:
nan_counts = df_brands.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_brands.shape)

Count of NaN values in each column:
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
_id.$oid          0
cpg.$id.$oid      0
cpg.$ref          0
brandCode       234
dtype: int64
(1167, 9)


In [192]:
df_brands = df_brands.drop(columns=['categoryCode','topBrand','category'])

In [193]:
nan_counts = df_brands.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_brands.shape)

Count of NaN values in each column:
barcode           0
name              0
_id.$oid          0
cpg.$id.$oid      0
cpg.$ref          0
brandCode       234
dtype: int64
(1167, 6)


In [194]:
import re

def generate_brand_code(name):
    stripped_name = re.sub(r'[^a-zA-Z0-9]', '', name).upper()
    return f"{stripped_name} @{name}"

# Update the brandCode for rows where it is NaN
df_brands.loc[df_brands['brandCode'].isna(), 'brandCode'] = df_brands.loc[df_brands['brandCode'].isna(), 'name'].apply(generate_brand_code)

In [195]:
nan_counts = df_brands.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_brands.shape)

Count of NaN values in each column:
barcode         0
name            0
_id.$oid        0
cpg.$id.$oid    0
cpg.$ref        0
brandCode       0
dtype: int64
(1167, 6)


In [196]:
nan_counts = df_users.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_users.shape)

Count of NaN values in each column:
active                0
role                  0
signUpSource         48
state                56
_id.$oid              0
createdDate.$date     0
lastLogin.$date      62
dtype: int64
(495, 7)


In [197]:
df_users = df_users.drop(columns=['active', 'role','state', 'signUpSource','lastLogin.$date'])

In [198]:
nan_counts = df_users.isna().sum()

print("Count of NaN values in each column:")
print(nan_counts)
print(df_users.shape)

Count of NaN values in each column:
_id.$oid             0
createdDate.$date    0
dtype: int64
(495, 2)


In [199]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 632 entries, 0 to 1117
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   purchasedItemCount      632 non-null    float64
 1   rewardsReceiptItemList  632 non-null    object 
 2   rewardsReceiptStatus    632 non-null    object 
 3   totalSpent              632 non-null    object 
 4   userId                  632 non-null    object 
 5   _id.$oid                632 non-null    object 
 6   createDate.$date        632 non-null    int64  
 7   dateScanned.$date       632 non-null    int64  
 8   modifyDate.$date        632 non-null    int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 49.4+ KB


In [200]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   _id.$oid           495 non-null    object
 1   createdDate.$date  495 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [201]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   name          1167 non-null   object
 2   _id.$oid      1167 non-null   object
 3   cpg.$id.$oid  1167 non-null   object
 4   cpg.$ref      1167 non-null   object
 5   brandCode     1167 non-null   object
dtypes: object(6)
memory usage: 54.8+ KB


In [202]:
def create_dim_date(start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    dim_date = pd.DataFrame(date_range, columns=['date'])
    dim_date['dateId'] = dim_date['date'].dt.strftime('%Y%m%d').astype(int)
    dim_date['year'] = dim_date['date'].dt.year
    dim_date['month'] = dim_date['date'].dt.month
    dim_date['day'] = dim_date['date'].dt.day
    dim_date['weekday'] = dim_date['date'].dt.weekday
    dim_date['week'] = dim_date['date'].dt.isocalendar().week
    dim_date['quarter'] = dim_date['date'].dt.quarter
    return dim_date

In [203]:
df_receipts['dateScanned.$date']

0       1609687531000
1       1609687483000
2       1609687537000
3       1609687534000
4       1609687506000
            ...      
1106    1614576748000
1112    1614560548000
1113    1614594748000
1114    1614594147000
1117    1614604048000
Name: dateScanned.$date, Length: 632, dtype: int64

In [204]:
df_users['createdDate.$date']

0      1609687444800
1      1609687444800
2      1609687444800
3      1609687530554
4      1609687444800
           ...      
490    1418998882381
491    1418998882381
492    1418998882381
493    1418998882381
494    1418998882381
Name: createdDate.$date, Length: 495, dtype: int64

In [205]:
# Create DimDate table
start_date = min(pd.to_datetime(df_receipts['dateScanned.$date'], unit='ms').min(),
                 pd.to_datetime(df_users['createdDate.$date'], unit='ms').min())
end_date = max(pd.to_datetime(df_receipts['dateScanned.$date'], unit='ms').max(),
               pd.to_datetime(df_users['createdDate.$date'], unit='ms').max())
df_dim_date = create_dim_date(start_date, end_date)

df_dim_date

,date,dateId,year,month,day,weekday,week,quarter
0,2014-12-19 14:21:22.381,20141219,2014,12,19,4,51,4
1,2014-12-20 14:21:22.381,20141220,2014,12,20,5,51,4
2,2014-12-21 14:21:22.381,20141221,2014,12,21,6,51,4
3,2014-12-22 14:21:22.381,20141222,2014,12,22,0,52,4
4,2014-12-23 14:21:22.381,20141223,2014,12,23,1,52,4
...,...,...,...,...,...,...,...,...
2260,2021-02-25 14:21:22.381,20210225,2021,2,25,3,8,1
2261,2021-02-26 14:21:22.381,20210226,2021,2,26,4,8,1
2262,2021-02-27 14:21:22.381,20210227,2021,2,27,5,8,1
2263,2021-02-28 14:21:22.381,20210228,2021,2,28,6,8,1


In [206]:
df_receipts.shape

(632, 9)

In [207]:
'''
for index, row in df_receipts.iterrows():
    if not pd.isna(row['rewardsReceiptItemList']):
        items = row['rewardsReceiptItemList']
        if isinstance(items, list):
            for item in items:
                receipt_items.append({
                    'receiptId': row['_id.$oid'],
                    'description': item.get('description', 'Unknown'),
                    'quantity': item.get('quantityPurchased', 0),
                    'price': item.get('finalPrice', 0.0)
                })'''
for receipt in receipts_data:
    if 'rewardsReceiptItemList' in receipt:
        for item in receipt['rewardsReceiptItemList']:
            print(item)
            # receipt_items.append({
            #     'receiptId': receipt['_id']['$oid'],
            #     'description': item.get('description', 'Unknown'),
            #     'quantity': item.get('quantityPurchased', 0),
            #     'price': item.get('finalPrice', 0.0)
            # })

# df_fact_receipt_items = pd.DataFrame(receipt_items)

# Assuming brandCode can be mapped from description or another mechanism
# This example assumes brandCode is somehow derived or matched
# df_fact_receipt_items

Streaming output truncated to the last 5000 lines.
{'barcode': '078742372198', 'description': 'GV PWD 2LB', 'discountedItemPrice': '1.62', 'finalPrice': '1.62', 'itemNumber': '078742372198', 'itemPrice': '1.62', 'originalMetaBriteBarcode': '', 'originalReceiptItemText': 'GV PWD 2LB', 'partnerItemId': '1276', 'quantityPurchased': 1}
{'brandCode': 'HY-VEE', 'description': 'HYV GRADE A X LRG EG', 'discountedItemPrice': '1.29', 'finalPrice': '1.29', 'itemPrice': '1.29', 'originalReceiptItemText': 'HYV GRADE A X LRG EG', 'partnerItemId': '1040', 'quantityPurchased': 1}
{'brandCode': 'HY-VEE', 'description': 'HYV LF BLUEBERRY YOG', 'discountedItemPrice': '0.54', 'finalPrice': '0.54', 'itemPrice': '0.54', 'originalReceiptItemText': 'HYV LF BLUEBERRY YOG', 'partnerItemId': '1041', 'quantityPurchased': 1}
{'barcode': '036632011077', 'brandCode': 'LIGHT & FIT GREEK', 'competitiveProduct': True, 'description': 'Light & Fit Greek Crunch Key Lime Pie Yogurt', 'discountedItemPrice': '0.54', 'finalPr

In [208]:

receipt_items = []
for receipt in receipts_data:
    if 'rewardsReceiptItemList' in receipt:
        for item in receipt['rewardsReceiptItemList']:
            receipt_items.append({
                'receiptId': receipt['_id']['$oid'],
                'pointsPayerId': item.get('pointsPayerId', 'Unknown'),
                'rewardsProductPartnerId': item.get('pointsPayerId', 'Unknown'),
                'quantity': item.get('quantityPurchased', 0),
                'price': item.get('finalPrice', 0.0)
            })
df_fact_receipt_items = pd.DataFrame(receipt_items)
df_fact_receipt_items

,receiptId,pointsPayerId,rewardsProductPartnerId,quantity,price
0,5ff1e1eb0a720f0523000575,Unknown,Unknown,5,26.00
1,5ff1e1bb0a720f052300056b,Unknown,Unknown,1,1
2,5ff1e1bb0a720f052300056b,5332f5fbe4b03c9a25efd0ba,5332f5fbe4b03c9a25efd0ba,1,10.00
3,5ff1e1f10a720f052300057a,Unknown,Unknown,0,0.0
4,5ff1e1ee0a7214ada100056f,Unknown,Unknown,4,28.00
...,...,...,...,...,...
6936,603cc2bc0a720fde100003e9,Unknown,Unknown,1,11.99
6937,603cc0630a720fde100003e6,Unknown,Unknown,1,22.97
6938,603cc0630a720fde100003e6,Unknown,Unknown,1,11.99
6939,603ce7100a7217c72c000405,Unknown,Unknown,1,22.97


In [209]:
print(df_fact_receipt_items['pointsPayerId'].value_counts())
print(df_fact_receipt_items['rewardsProductPartnerId'].value_counts())

pointsPayerId
Unknown                     5674
559c2234e4b06aca36af13c6     331
5332f5f6e4b03c9a25efd0b4     281
5332f5fbe4b03c9a25efd0ba     230
550b2565e4b001d5e9e4146f     190
5332f709e4b03c9a25efd0f1      90
5332f5f3e4b03c9a25efd0ae      56
5a734034e4b0d58f376be874      31
5e825d64f221c312e698a62a      25
5eebc5412455c97a877ef382      24
53e10d6368abd3c7065097cc       3
5f036cedac99f02d72388690       2
5fb6b608be37ce522e165cb9       1
5f58fa84be37ce21f1287c0f       1
5e7cf838f221c312e698a628       1
5ffdb69ebe37ce746df1192a       1
Name: count, dtype: int64
rewardsProductPartnerId
Unknown                     5674
559c2234e4b06aca36af13c6     331
5332f5f6e4b03c9a25efd0b4     281
5332f5fbe4b03c9a25efd0ba     230
550b2565e4b001d5e9e4146f     190
5332f709e4b03c9a25efd0f1      90
5332f5f3e4b03c9a25efd0ae      56
5a734034e4b0d58f376be874      31
5e825d64f221c312e698a62a      25
5eebc5412455c97a877ef382      24
53e10d6368abd3c7065097cc       3
5f036cedac99f02d72388690       2
5fb6b608be37

In [210]:
df_brands.columns

Index(['barcode', 'name', '_id.$oid', 'cpg.$id.$oid', 'cpg.$ref', 'brandCode'], dtype='object')

In [211]:
df_brands[df_brands['cpg.$id.$oid']=='559c2234e4b06aca36af13c6']

,barcode,name,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
7,511111104810,J.L. Kraft,5cdad0f5166eb33eb7ce0faa,559c2234e4b06aca36af13c6,Cogs,J.L. KRAFT
12,511111201076,Calumet,588ba07be4b02187f85cdadd,559c2234e4b06aca36af13c6,Cogs,CALUMET
22,511111802129,Jack Daniel's,57ebc011e4b0ac389136a335,559c2234e4b06aca36af13c6,Cogs,JACK DANIEL'S BARBECUE
30,511111201915,Taco Bell,57ebc2e7e4b0ac389136a34b,559c2234e4b06aca36af13c6,Cogs,TACO BELL
35,511111100621,GODIVA Instant Pudding Mix,59dfaad1e4b0a56a2fa69abc,559c2234e4b06aca36af13c6,Cogs,GODIVA DRY PACKAGED DESSERTS
...,...,...,...,...,...,...
1136,511111601951,Polly-O,57ebc2b7e4b0ac389136a347,559c2234e4b06aca36af13c6,Cogs,POLLY-O
1137,511111202165,Gevalia Iced Coffee,57ebbfa9e4b0ac389136a331,559c2234e4b06aca36af13c6,Cogs,
1139,511111401209,Baker's,5887a051e4b02187f85cdad0,559c2234e4b06aca36af13c6,Cogs,BAKER'S
1154,511111302346,Crystal Light,57d95826e4b0ac389136a2b7,559c2234e4b06aca36af13c6,Cogs,CRYSTAL LIGHT


In [212]:
df_dim_date = df_dim_date[['dateId', 'date', 'year', 'month', 'day', 'weekday', 'week', 'quarter']]
df_dim_date

,dateId,date,year,month,day,weekday,week,quarter
0,20141219,2014-12-19 14:21:22.381,2014,12,19,4,51,4
1,20141220,2014-12-20 14:21:22.381,2014,12,20,5,51,4
2,20141221,2014-12-21 14:21:22.381,2014,12,21,6,51,4
3,20141222,2014-12-22 14:21:22.381,2014,12,22,0,52,4
4,20141223,2014-12-23 14:21:22.381,2014,12,23,1,52,4
...,...,...,...,...,...,...,...,...
2260,20210225,2021-02-25 14:21:22.381,2021,2,25,3,8,1
2261,20210226,2021-02-26 14:21:22.381,2021,2,26,4,8,1
2262,20210227,2021-02-27 14:21:22.381,2021,2,27,5,8,1
2263,20210228,2021-02-28 14:21:22.381,2021,2,28,6,8,1


In [213]:
'''# Renaming columns for consistency
df_users.rename(columns={'_id.$oid': 'userId'}, inplace=True)
df_brands.rename(columns={'_id.$oid': 'brandId'}, inplace=True)
df_receipts.rename(columns={'_id.$oid': 'receiptId', 'dateScanned.$date': 'dateScanned'}, inplace=True)

# Adding dateScannedId to FactReceipts
df_receipts['dateScanned'] = pd.to_datetime(df_receipts['dateScanned'], unit='ms')
df_receipts['dateScannedId'] = df_receipts['dateScanned'].dt.strftime('%Y%m%d').astype(int)

# Creating FactReceipts and dimension tables
df_fact_receipts = df_receipts[['receiptId', 'userId', 'dateScannedId', 'rewardsReceiptStatus', 'totalSpent', 'purchasedItemCount']]
df_dim_users = df_users[['userId', 'createdDate.$date']].rename(columns={'createdDate.$date': 'createdDateId'})
df_dim_brands = df_brands[['brandId', 'name', 'brandCode']]'''

"# Renaming columns for consistency\ndf_users.rename(columns={'_id.$oid': 'userId'}, inplace=True)\ndf_brands.rename(columns={'_id.$oid': 'brandId'}, inplace=True)\ndf_receipts.rename(columns={'_id.$oid': 'receiptId', 'dateScanned.$date': 'dateScanned'}, inplace=True)\n\n# Adding dateScannedId to FactReceipts\ndf_receipts['dateScanned'] = pd.to_datetime(df_receipts['dateScanned'], unit='ms')\ndf_receipts['dateScannedId'] = df_receipts['dateScanned'].dt.strftime('%Y%m%d').astype(int)\n\n# Creating FactReceipts and dimension tables\ndf_fact_receipts = df_receipts[['receiptId', 'userId', 'dateScannedId', 'rewardsReceiptStatus', 'totalSpent', 'purchasedItemCount']]\ndf_dim_users = df_users[['userId', 'createdDate.$date']].rename(columns={'createdDate.$date': 'createdDateId'})\ndf_dim_brands = df_brands[['brandId', 'name', 'brandCode']]"

In [214]:
conn = sqlite3.connect('/content/receipts.db')
c = conn.cursor()

# Creating DimDate table
df_dim_date.to_sql('DimDate', conn, if_exists='replace', index=False)

# Creating DimUsers table
df_users.rename(columns={'_id.$oid': 'userId', 'createdDate.$date': 'createdDate'}, inplace=True)
df_users['createdDateId'] = df_users['createdDate'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y%m%d')).astype(int)
df_users = df_users[['userId', 'createdDateId']]
df_users.to_sql('DimUsers', conn, if_exists='replace', index=False)

# Creating DimBrands table
df_brands.rename(columns={'_id.$oid': 'brandId'}, inplace=True)
df_brands = df_brands[['brandId', 'name', 'brandCode']]
df_brands.to_sql('DimBrands', conn, if_exists='replace', index=False)

# Creating FactReceipts table
df_receipts.rename(columns={'_id.$oid': 'receiptId', 'dateScanned.$date': 'dateScanned'}, inplace=True)
df_receipts['dateScannedId'] = df_receipts['dateScanned'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y%m%d')).astype(int)
df_fact_receipts = df_receipts[['receiptId', 'userId', 'dateScannedId', 'rewardsReceiptStatus', 'totalSpent', 'purchasedItemCount']]
df_fact_receipts.to_sql('FactReceipts', conn, if_exists='replace', index=False)

# Merging to create FactReceiptItems table

df_fact_receipt_items = df_fact_receipt_items.merge(df_brands[['brandId', 'brandCode']], left_on='pointsPayerId', right_on='brandCode', how='left')
df_fact_receipt_items = df_fact_receipt_items[['receiptId', 'brandId', 'quantity', 'price']]
df_fact_receipt_items.to_sql('FactReceiptItems', conn, if_exists='replace', index=False)

# Committing and closing the connection
conn.commit()
conn.close()

In [215]:
# Connecting to SQLite database
conn = sqlite3.connect('/content/receipts.db')
c = conn.cursor()


def print_table_contents(table_name):
    print(f"\nContents of {table_name}:")
    for row in c.execute(f"SELECT * FROM {table_name} LIMIT 5"):
        print(row)
# Print contents of tables
print_table_contents('FactReceipts')
print_table_contents('DimUsers')
print_table_contents('DimBrands')
print_table_contents('DimDate')

# Simplified queries to test
print("\nMost recent date in DimDate:")
c.execute("SELECT MAX(date) FROM DimDate")
print(c.fetchone())

print("\nCount of receipts in FactReceipts:")
c.execute("SELECT COUNT(*) FROM FactReceipts")
print(c.fetchone())

print("\nCount of users in DimUsers:")
c.execute("SELECT COUNT(*) FROM DimUsers")
print(c.fetchone())

print("\nCount of brands in DimBrands:")
c.execute("SELECT COUNT(*) FROM DimBrands")
print(c.fetchone())

conn.close()


Contents of FactReceipts:
('5ff1e1eb0a720f0523000575', '5ff1e1eacfcf6c399c274ae6', 20210103, 'FINISHED', '26.00', 5.0)
('5ff1e1bb0a720f052300056b', '5ff1e194b6a9d73a3a9f1052', 20210103, 'FINISHED', '11.00', 2.0)
('5ff1e1f10a720f052300057a', '5ff1e1f1cfcf6c399c274b0b', 20210103, 'REJECTED', '10.00', 1.0)
('5ff1e1ee0a7214ada100056f', '5ff1e1eacfcf6c399c274ae6', 20210103, 'FINISHED', '28.00', 4.0)
('5ff1e1d20a7214ada1000561', '5ff1e194b6a9d73a3a9f1052', 20210103, 'FINISHED', '1.00', 2.0)

Contents of DimUsers:
('5ff1e194b6a9d73a3a9f1052', 20210103)
('5ff1e194b6a9d73a3a9f1052', 20210103)
('5ff1e194b6a9d73a3a9f1052', 20210103)
('5ff1e1eacfcf6c399c274ae6', 20210103)
('5ff1e194b6a9d73a3a9f1052', 20210103)

Contents of DimBrands:
('601ac115be37ce2ead437551', 'test brand @1612366101024', 'TESTBRAND1612366101024 @test brand @1612366101024')
('601c5460be37ce2ead43755f', 'Starbucks', 'STARBUCKS')
('601ac142be37ce2ead43755d', 'test brand @1612366146176', 'TEST BRANDCODE @1612366146176')
('601ac142

In [217]:
# Reconnect to the SQLite database
conn = sqlite3.connect('/content/receipts.db')
c = conn.cursor()

# Define the queries
queries = {
    "Top 5 brands by receipts scanned for the most recent month":
    """
    SELECT b.name, COUNT(*) as scan_count
    FROM FactReceipts fr
    JOIN DimDate dd ON fr.dateScannedId = dd.dateId
    JOIN FactReceiptItems fri ON fr.receiptId = fri.receiptId
    JOIN DimBrands b ON fri.brandId = b.brandId
    WHERE dd.date >= DATE('now', '-1 month')
    GROUP BY b.name
    ORDER BY scan_count DESC
    LIMIT 5
    """,

    "Ranking of the top 5 brands by receipts scanned for the previous month":
    """
    SELECT b.name, COUNT(*) as scan_count
    FROM FactReceipts fr
    JOIN DimDate dd ON fr.dateScannedId = dd.dateId
    JOIN FactReceiptItems fri ON fr.receiptId = fri.receiptId
    JOIN DimBrands b ON fri.brandId = b.brandId
    WHERE dd.date >= DATE('now', '-2 months') AND dd.date < DATE('now', '-1 month')
    GROUP BY b.name
    ORDER BY scan_count DESC
    LIMIT 5
    """,

    "Average spend from receipts with 'Accepted' or 'Rejected' status":
    """
    SELECT rewardsReceiptStatus, AVG(totalSpent) as avg_spent
    FROM FactReceipts
    WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
    GROUP BY rewardsReceiptStatus
    """,

    "Total number of items purchased from receipts with 'Accepted' or 'Rejected' status":
    """
    SELECT fr.rewardsReceiptStatus, SUM(fri.quantity) as total_items
    FROM FactReceipts fr
    JOIN FactReceiptItems fri ON fr.receiptId = fri.receiptId
    WHERE fr.rewardsReceiptStatus IN ('Accepted', 'Rejected')
    GROUP BY fr.rewardsReceiptStatus
    """,

    "Brand with the most spend among users created within the past 6 months":
    """
    SELECT b.name, SUM(fri.price * fri.quantity) as total_spent
    FROM FactReceipts fr
    JOIN DimUsers du ON fr.userId = du.userId
    JOIN FactReceiptItems fri ON fr.receiptId = fri.receiptId
    JOIN DimBrands b ON fri.brandId = b.brandId
    WHERE du.createdDateId >= strftime('%Y%m%d', 'now', '-6 months')
    GROUP BY b.name
    ORDER BY total_spent DESC
    LIMIT 1
    """,

    "Brand with the most transactions among users created within the past 6 months":
    """
    SELECT b.name, COUNT(*) as transaction_count
    FROM FactReceipts fr
    JOIN DimUsers du ON fr.userId = du.userId
    JOIN FactReceiptItems fri ON fr.receiptId = fri.receiptId
    JOIN DimBrands b ON fri.brandId = b.brandId
    WHERE du.createdDateId >= strftime('%Y%m%d', 'now', '-6 months')
    GROUP BY b.name
    ORDER BY transaction_count DESC
    LIMIT 1
    """
}

# Execute and display the results of the queries
for query_name, query in queries.items():
    print(f"\n{query_name}:")
    for row in c.execute(query):
        print(row)

# Close the connection
conn.close()



Top 5 brands by receipts scanned for the most recent month:

Ranking of the top 5 brands by receipts scanned for the previous month:

Average spend from receipts with 'Accepted' or 'Rejected' status:

Total number of items purchased from receipts with 'Accepted' or 'Rejected' status:

Brand with the most spend among users created within the past 6 months:

Brand with the most transactions among users created within the past 6 months:
